# **Compute Sentiment from 4 SyuzhetR and 7 SentimentR Models**

* https://www.youtube.com/watch?v=U3ByGh8RmSc

* https://github.com/ttimbers/intro-to-reticulate

[Use R on Google Colab!](https://colab.research.google.com/notebook#create=true&language=r)

# Configure Notebook

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Connect to Google gDrive

* https://towardsdatascience.com/access-google-drive-using-google-colab-running-an-r-kernel-3736db7835


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
%cd ./MyDrive/research/2022/sentiment_arcs/notebooks/

In [ ]:
!ls

# Define Globals

In [ ]:
# CUSTOMIZE: define subdirectory paths

# Input subdirectories
subdir_novels = '../in1_novels/'
subdir_preprocessed = '../in2_preprocessed_text/'
subdir_sentiments = '../in3_sentiments/'

# data_raw_subdir  = './data/sentiments_raw/'
# data_clean_subdir  = './data/sentiments_clean/'
# novels_subdir = './in1_novels/'
# lexicons_subdir = './lexicons/'

# Output subdirectories
subdir_plots_arcs = '../out1_arcs/'
subdir_plots_cruxes = '../out2_cruxes/'

# plots_subdir = './out1_arcs/'
# plots_subdir = '../book_shape_of_stories/'

# crux_subdir = './crux/'
# code_subdir  = './sentiment_arcs/sentiment_arcs/'

In [ ]:
# Consider requiring this as header line in source text files so all data in one place

novels_dt = {
  'cdickens_achristmascarol':['A Christmas Carol by Charles Dickens ',1843,1399],
  'cdickens_greatexpectations':['Great Expectations by Charles Dickens' ,1861, 7230],
  'dbrown_thedavincicode':['The Da Vinci Code by Dan Brown', 2003, 9475],
  'ddefoe_robinsoncrusoe':['Robinson Crusoe by Daniel Defoe',1719, 2280],
  'eljames_fiftyshadesofgrey':['Fifty Shades of Grey by E.L. James', 2011, 8184],
  'emforster_howardsend':['Howards End by E.M. Forester', 1910, 8999],
  'fbaum_thewonderfulwizardofoz':['The Wonderful Wizard of Oz by Frank Baum', 1850, 2238],
  'fdouglass_narrativelifeofaslave':['Narrative of the life of Frederick Douglass, an American Slave by Frederick Douglass', 1845, 1688],
  'fscottfitzgerald_thegreatgatsby':['The Great Gatsby by F. Scott Fitzgerald', 1925, 2950],
  'geliot_middlemarch':['Middlemarch by George Eliot', 1871, 10373],
  'hjames_portraitofalady':['The Portrait of a Lady by Henry James', 1881, 13258],
  'homer-ewilson_odyssey':['The Odyssey by Homer (trans Emily Wilson)', 2018, 6814],
  'imcewan_machineslikeme':['Machines Like Me by Ian McEwan', 2019, 6448],
  'jausten_prideandprejudice':['Pride and Prejudice by Jane Austen', 1813, 5891],
  'jconrad_heartofdarkness':['Heart of Darkness by Joseph Conrad', 1902, 1619],
  'jjoyce_portraitoftheartist':['A Portrait of the Artist as a Young Man by James Joyce', 2016, 5584],
  'jkrowling_1sorcerersstone':['Harry Potter and the Sorcerer’s Stone by J.K. Rowling', 1997, 5488],
  'jkrowling_4gobletoffire':['Harry Potter and the Goblet of Fire by J.K. Rowling', 2000, 6809],
  'jkrowling_4gobletoffire_screenplay':['Screenplay for Harry Potter and the Goblet of Fire by J.K. Rowling and Steve Klovesand J.', 2005, 582],
  # 'jkrowling_7deathlyhollows':['Harry Potter and the Deathly Hallows by J.K. Rowling', 2007, 6588],
  'kvonnegut_slaughterhousefive':['Slaughterhouse Five by Kurt Vonnegut', 1969, 5607],
  'mproust-mtreharne_3guermantesway':['In Search of Lost Time, Vol 3: The Guermantes Way by Marcel Proust', 1920, 8388],
  'mshelley_frankenstein':['Frankenstein by Mary Shelly', 1818, 3282],
  'mtwain_huckleberryfinn':['Huckleberry Finn by Mark Twain', 1884, 5775],
  'pjackson_thelightningthief':["The Lightning Thief by Peter Jackson", 2005, 8277],
  'staugustine_confessions9end':['Confessions (Books 1-9) by St. Augustine', 400, 3673],
  'tmorrison_beloved':['Beloved by Toni Morrison', 1987, 7102],
  'vnabokov_palefire':['Pale Fire by Viktor Nabokov', 1962, 2984],
  'vwoolf_mrsdalloway':['Mrs. Dalloway by Virginia Woolf', 1925, 3647],
  'vwoolf_orlando':['Orlando by Virginia Woolf', 1928, 2,992],
  'vwoolf_thewaves':['The Waves by Virginia Woolf', 1931, 3919],
  'vwoolf_tothelighthouse':['To The Lighthouse by Virginia Woolf', 1927, 3403],
  'wgolding_lordoftheflies':['Lord of the Flies by William Golding', 1954, 5441]
}

# R Libraries: Install and Load

In [ ]:
# !pip install rpy2

In [ ]:
# !pip install -U rpy2

In [ ]:
# enables the %%R magic, not necessary if you've already done this
%load_ext rpy2.ipython

In [ ]:
# %reload_ext rpy2.ipython

In [ ]:
%%time 
%%capture 
%%R

# Install Syuzhet.R and Sentiment.R

install.packages(c('syuzhet', 'sentimentr', 'tidyverse', 'lexicon'))

library(syuzhet)
library(sentimentr)
library(tidyverse)
library(lexicon)

# NOTE:   17s
#       1m05s

In [ ]:
# %reload_ext rpy2.ipython

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

In [ ]:
%%R

R.version.string

In [ ]:
%%R

sessionInfo()

In [ ]:
%%R

Sys.getenv

# Python Libraries: Install

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from glob import glob
import copy
import json

In [ ]:
# Show all default settings

# plt.rcParams # all
# plt.rcParams["figure.figsize"]

# Default figure size
fig_size = plt.rcParams.get('figure.figsize')

print(" The Default figure size in jupyter notebook is: ", fig_size)

In [ ]:
# Set matplotlib plot figure.figsize

new_plt_size = plt.rcParams["figure.figsize"]=(20,10)

print(" New figure size: ",new_plt_size)

# Python Utility Functions

In [ ]:
# Utility functions to read/write nested Dictionary (key=novel) of DataFrames (Cols = Model Sentiment Series) 

def write_dict_dfs(adict, out_file='sentiments.json', out_dir=subdir_sentiments):
  '''
  Given a Dictionary of DataFrames and optional output filename and output directory
  Write as nested json file
  '''

  # convert dataframes into dictionaries
  data_dict = {
      key: adict[key].to_dict(orient='records') 
      for key in adict.keys()
  }

  # write to disk
  out_fullpath = f'{out_dir}{out_file}'
  print(f'Saving file to: {out_fullpath}')
  with open(out_fullpath, 'w') as fp:
    json.dump(
      data_dict, 
      fp, 
      indent=4, 
      sort_keys=True
    )

  return 

def read_dict_dfs(in_file='sentiments.json', in_dir=subdir_sentiments):
  '''
  Given a Dictionary of DataFrames and optional output filename and output directory
  Read nested json file into Dictionary of DataFrames
  '''

  # read from disk
  in_fullpath = f'{in_dir}{in_file}'
  with open(in_fullpath, 'r') as fp:
      data_dict = json.load(fp)

  # convert dictionaries into dataframes
  all_dt = {
      key: pd.DataFrame(data_dict[key]) 
      for key in data_dict
  }

  return all_dt

# Read all Preprocessed Novels

In [ ]:
# Create a List (preprocessed_ls) of all preprocessed text files

try:
    preprocessed_ls = glob(f'{subdir_preprocessed}*.csv')
    preprocessed_ls = [x.split('/')[-1] for x in preprocessed_ls]
    preprocessed_ls = [x.split('.')[0] for x in preprocessed_ls]
except IndexError:
    raise RuntimeError('No csv file found')

print('\n'.join(preprocessed_ls))
print('\n')
print(f'Found {len(preprocessed_ls)} Preprocessed files in {subdir_preprocessed}')

In [ ]:
# Read all preprocessed text files into master DataFrame (novels_all_dt)

novels_all_dt = {}

for i,anovel in enumerate(preprocessed_ls):
  print(f'Processing #{i}: {anovel}...')
  afile_fullpath = f'{subdir_preprocessed}{anovel}.csv'
  print(f'               {afile_fullpath}')
  anovel_df = pd.read_csv(afile_fullpath)
  novels_all_dt[anovel] = anovel_df

In [ ]:
# Verify the novels read into master Dictionary of DataFrames

novels_all_dt.keys()
print('\n')
print(f'There were {len(novels_all_dt)} preprocessed novels read into the Dict novels_all_dt')

In [ ]:
# Check if there are any Null strings in the text_clean columns

for i, anovel in enumerate(list(novels_all_dt.keys())):
  print(f'\nNovel #{i}: {anovel}')
  nan_ct = novels_all_dt[anovel].text_clean.isna().sum()
  if nan_ct > 0:
    print(f'      {nan_ct} Null strings in the text_clean column')

In [ ]:
# Fill in all the Null value of text_clean with placeholder 'empty_string'

for i, anovel in enumerate(list(novels_all_dt.keys())):
  # print(f'Novel #{i}: {anovel}')
  # Fill all text_clean == Null with 'empty_string' so sentimentr::sentiment doesn't break
  novels_all_dt[anovel][novels_all_dt[anovel].text_clean.isna()] = 'empty_string'

In [ ]:
# Verify one DataFrame in the master Dictionary

novels_all_dt['dbrown_thedavincicode'].head()

**[SKIP] to next section**

In [ ]:
%%R -i df -w 5 -h 5 --units in -r 200
# import df from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution

install.packages("ggplot2", repos='http://cran.us.r-project.org', quiet=TRUE)
library(ggplot2)
ggplot(df, aes(x=cups_of_coffee, y=productivity)) + geom_line()

In [ ]:
from pathlib import Path

In [ ]:
!python --version

In [ ]:
if (path := next(Path(subdir_preprocessed).glob("*.csv"), None)) is None:
    print("No .csv file present!")
else:
    print(path)

In [ ]:
try:
    path_ls = next(Path(subdir_preprocessed).glob("*.csv"))
except StopIteration:
    raise RuntimeError("No .csv file present!")

path_ls

In [ ]:
import pathlib
[p for p in pathlib.Path('../in2_preprocessed_text/').iterdir() if p.is_file()]

# [SKIP] SyuzhetR Library

* https://github.com/mjockers/syuzhet

* https://cran.r-project.org/web/packages/syuzhet/index.html

In [ ]:
preprocessed_ls

In [ ]:
novels_all_dt['cdickens_achristmascarol']

In [ ]:
import rpy2.robjects as robjects
a = robjects.r('list(foo="barbat", fizz=123)')

In [ ]:
test_df = novels_all_dt['cdickens_achristmascarol']
test_df.head()

In [ ]:
%%R -i test_df -o get_syuzhetr_syuzhet

get_syuzhetr_syuzhet <- function(test_df) {
    sentiment_syuzhet <- get_sentiment(test_df$text_clean, method='syuzhet')
}

In [ ]:
%%R -i test_df
get_syuzhetr_syuzhet(test_df)

In [ ]:
%%R -i test_df

require(dplyr)
glimpse(test_df)

In [ ]:
%%R -i novels_all_dt -o novels_all_dt
# import df from global environment
# make default figure size 5 by 5 inches with 200 dpi resolution

install.packages("ggplot2", repos='http://cran.us.r-project.org', quiet=TRUE)
library(ggplot2)
ggplot(df, aes(x=cups_of_coffee, y=productivity)) + geom_line()

In [ ]:
from rpy2.robjects.packages import importr
base = importr('base')
stats = importr('stats')
graphics = importr('graphics')

plot = graphics.plot
# rnorm = stats.rnorm
# plot(rnorm(100), ylab="random")

# Get Sentiments using 4 SyuzhetR Models on Every Novel

In [ ]:
# Make a copy of DataFrame for 4 SyuzhetR Models

novels_syuzhetr_dt = copy.deepcopy(novels_all_dt)

## Option (a): Read Previously Computed SyuzhetR Values from Datafiles

In [ ]:
# Read in Saved SyuzhetR Datafile from subdir_sentiments/all_4syuzhetr.json

novels_syuzhetr_dt = read_dict_dfs('all_4syuzhetr.json')
novels_syuzhetr_dt.keys()

In [ ]:
# Verify all the Novels have 4 Syuzhet Model Values

for i, anovel in enumerate(list(novels_syuzhetr_dt.keys())):
  print(f'Novel #{i}: {anovel}')
  novels_syuzhetr_dt[anovel].drop(columns=['Unnamed: 0'], inplace=True)
  print(f'      df.shape: {novels_syuzhetr_dt[anovel].shape}')

In [ ]:
# Verify DataFrame for test novel

novel_str = 'cdickens_achristmascarol'
novels_syuzhetr_dt[novel_str].head()

## Option (b): Compute New SyuzhetR Values

In [ ]:
%whos dict


In [ ]:
novels_all_dt['cdickens_achristmascarol']['text_clean'].to_list()

In [ ]:
# Compute Sentiments from all 4 Syuzhet Models applied to all 32 Novels (4 x 32 = 128 runs)

# NOTE: 9m45s 23:30 on 20220114 Colab Pro (33 Novels)

# base = importr('base')
syuzhet = importr('syuzhet')

# base.rank(0, na_last = True)
novels_keys_ls = list(novels_all_dt.keys())
novels_keys_ls.sort()
for i, anovel in enumerate(novels_keys_ls):
  print(f'Processing Novel #{i}: {anovel}...')
  novels_syuzhetr_dt[anovel]['syuzhetr_syuzhet'] = syuzhet.get_sentiment(novels_syuzhetr_dt[anovel]['text_clean'].to_list(), method='syuzhet')
  novels_syuzhetr_dt[anovel]['syuzhetr_bing'] = syuzhet.get_sentiment(novels_syuzhetr_dt[anovel]['text_clean'].to_list(), method='bing')
  novels_syuzhetr_dt[anovel]['syuzhetr_afinn'] = syuzhet.get_sentiment(novels_syuzhetr_dt[anovel]['text_clean'].to_list(), method='afinn')
  novels_syuzhetr_dt[anovel]['syuzhetr_nrc'] = syuzhet.get_sentiment(novels_syuzhetr_dt[anovel]['text_clean'].to_list(), method='nrc')

### Save Checkpoint

In [ ]:
# Verify save_to directory

subdir_sentiments
print('\n')
!ls $subdir_sentiments

In [ ]:
# Save sentiment values to subdir_sentiments

write_dict_dfs(novels_syuzhetr_dt, out_file='all_4syuzhetr.json', out_dir=subdir_sentiments)

## Plot SyuzhetR 4 Models

In [ ]:
#@markdown Select option to save plots:
Save_Raw_Plots = True #@param {type:"boolean"}

Save_Smooth_Plots = True #@param {type:"boolean"}
Resolution = "300" #@param ["100", "300"]



In [ ]:
syuzhetr_cols_ls = novels_syuzhetr_dt['cdickens_achristmascarol'].columns
syuzhetr_models_ls = [x for x in syuzhetr_cols_ls if 'syuzhetr_' in x]
syuzhetr_models_ls

# OR

# syuzhetr_models_ls = ['syuzhetr_afinn', 'syuzhetr_bing', 'syuzhetr_nrc', 'syuzhetr_syuzhet']

In [ ]:
novels_dt['cdickens_achristmascarol'][0]

In [ ]:
subdir_plots_arcs

In [ ]:
# Verify 4 Sentiment Models from Syuzhet for sample Novel

for i, anovel in enumerate(list(novels_all_dt.keys())):

  # Raw Sentiments 
  fig = novels_syuzhetr_dt[anovel][syuzhetr_models_ls].plot(title=f'{novels_dt[anovel][0]}\n SyuzhetR 4 Models: Raw Sentiments', alpha=0.3)
  plt.show()

  if Save_Raw_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_syuzhetr_raw_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))

  
  # Smoothed Sentiments (SMA 10%)
  # novel_sample = 'cdickens_achristmascarol'
  win_10per = int(novels_syuzhetr_dt[anovel].shape[0] * 0.1)
  novels_syuzhetr_dt[anovel][syuzhetr_models_ls].rolling(win_10per, center=True, min_periods=0).mean().plot(title=f'{novels_dt[anovel][0]}\n SyuzhetR 4 Models: Smoothed Sentiments (SMA 10%)', alpha=0.3)
  plt.show()

  if Save_Smooth_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_syuzhetr_smooth10sma_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))


# Get Sentiments using 7 SentimentR Models on Every Novel

In [ ]:
# Make a copy of DataFrame for 7 SentimentR Models

novels_sentimentr_dt = copy.deepcopy(novels_all_dt)


In [ ]:
# import rpy2

In [ ]:
# dir(rpy2.robjects)

In [ ]:
# !pip install -U rpy2

In [ ]:
# dir(lexicon)

In [ ]:
# from rpy2.robjects.packages import importr

# NOTE: 1m22s for 1 out of (7x32 = 224)

sentimentr = importr('sentimentr')
lexicon = importr('lexicon')

novels_keys_ls = list(novels_all_dt.keys())
# lexicon_robj = lexicon.hash_sentiment_huliu
for i, anovel in enumerate(novels_keys_ls[:1]):
  print(f'Processing Novel #{i}: {anovel}...')
  print( '                 jockers_rinker')
  novels_all_dt[anovel]['sentimentr_jockersrinker'] = novels_all_dt[anovel]['text_clean'].apply(lambda x: sentimentr.sentiment(x, polarity_dt=lexicon.hash_sentiment_jockers_rinker)) # polarity_dt=lexicon.hash_sentiment_jockers_rinker))

  print( '                 jockers')  # 1m20s
  ## novels_all_dt[anovel]['sentimentr_jockers'] = novels_all_dt[anovel]['text_clean'].apply(lambda x: sentimentr.sentiment(x, polarity_dt=lexicon.hash_sentiment_jockers)) # polarity_dt=lexicon.hash_sentiment_jockers_rinker))

  # lexicon_robj = lexicon.hash_sentiment_huliu
  print( '                 huliu')
  # novels_all_dt[anovel]['sentimentr_huliu'] = novels_all_dt[anovel]['text_clean'].apply(lambda x: sentimentr.sentiment(x, polarity_dt=lexicon.hash_sentiment_huliu)) # lexicon_robj)) # polarity_dt=lexicon.hash_sentiment_huliu))

  # lexicon_robj = lexicon.hash_sentiment_nrc
  print( '                 nrc')
  # novels_all_dt[anovel]['sentimentr_nrc'] = novels_all_dt[anovel]['text_clean'].apply(lambda x: sentimentr.sentiment(x, polarity_dt=lexicon_robj)) # polarity_dt=lexicon.hash_sentiment_nrc))

  print( '                 senticnet')
  # novels_all_dt[anovel]['sentimentr_senticnet'] = novels_all_dt[anovel]['text_clean'].apply(lambda x: sentimentr.sentiment(x, polarity_dt=lexicon_robj)[[3]]) # polarity_dt=lexicon.hash_sentiment_senticnet))

  print( '                 sentiword')
  # novels_all_dt[anovel]['sentimentr_sentiword'] = novels_all_dt[anovel]['text_clean'].apply(lambda x: sentimentr.sentiment(x, polarity_dt=lexicon_robj)) # polarity_dt=lexicon.hash_sentiment_sentiword))

  print( '                 loughran_mcdonald')
  # novels_all_dt[anovel]['sentimentr_loughran_mcdonald'] = novels_all_dt[anovel]['text_clean'].apply(lambda x: sentimentr.sentiment(x, polarity_dt=lexicon_robj)) # polarity_dt=lexicon.hash_sentiment_loughran_mcdonald))

  # test_sent = 'I love lint very much'
  # test_str = sentimentr.sentiment(test_sent)
  # print(f'SentimentR: {test_str} for \n            {test_sent}')
  # novels_all_dt[anovel]['sentimentr_jockersrinker'] = sentimentr.sentiment(novels_all_dt[anovel]['text_clean'].to_list())

In [ ]:
novels_all_dt['cdickens_achristmascarol'].head()

In [ ]:
from rpy2.robjects.packages import importr

sentiment = importr('sentimentr')

novels_keys_ls = list(novels_all_dt.keys())
for i, anovel in enumerate(novels_keys_ls[:1]):
  print(f'Processing Novel #{i}: {anovel}...')
  novels_all_dt[anovel]['sentimentr_jockersrinker'] = sentiment(novels_all_dt[anovel]['text_clean'].to_list(), polarity_dt=lexicon::hash_sentiment_jockers_rinker, 
                                                                hypen="", amplifier_weight=0.8, n_before=5, n_after=2,
                                                                adversative_weight=0.25, neutral_nonverb_like = FALSE, missing_value = 0)


In [ ]:
SentimentAnalysis <- apply(analyzeSentiment(s_v)[c('SentimentGI', 'SentimentLM', 'SentimentQDAP') ], 2, round, 2)
colnames(SentimentAnalysis) <- gsub('^Sentiment', "SA_", colnames(SentimentAnalysis))

In [ ]:
sentimentr_jockersrinker <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers_rinker, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_jockersrinker

In [ ]:
%%R

sessionInfo()

In [ ]:
test_ls = novels_all_dt['cdickens_achristmascarol']['text_clean'].to_list()
len(test_ls)

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr

In [ ]:
s_v = robjects.StrVector(test_ls)
type(s_v)

lexicon_name = lexicon::hash_sentiment_jockers_rinker

In [ ]:
novels_sentimentr_df = pd.DataFrame()

In [ ]:
novels_roots_ls = list(novels_all_dt.keys())

In [ ]:
for i,anovel_root in enumerate(novels_roots_ls):
  print(f'Novel #{i}: {anovel_root}')
  print(f'     {novels_all_dt[anovel_root].shape}')

In [ ]:
%%R -i novels_roots_ls

# novels_key_ls = c('cdickens_achristmascarol')

for(i in 1:length(novels_roots_ls)) {
  {print(novels_roots_ls[i])}
}

## Option (a): Read Previous Computed SentimentR Values from DataFile

In [ ]:
# Read in Saved SyuzhetR Datafile from subdir_sentiments/all_4syuzhetr.json

novels_sentimentr_dt = read_dict_dfs('all_7sentimentr.json')
novels_sentimentr_dt.keys()

In [ ]:
# Verify all the Novels have 4 Syuzhet Model Values

for i, anovel in enumerate(list(novels_sentimentr_dt.keys())):
  print(f'Novel #{i}: {anovel}')
  novels_synovels_sentimentr_dtuzhetr_dt[anovel].drop(columns=['Unnamed: 0'], inplace=True)
  print(f'      df.shape: {novels_sentimentr_dt[anovel].shape}')

In [ ]:
# Verify DataFrame for test novel

novel_str = 'cdickens_achristmascarol'
novels_sentimentr_dt[novel_str].head()

## Option (b): Compute New SentimentR Values

Call function in external get_sentimentr.R from within Python Loop

* https://medium.com/analytics-vidhya/calling-r-from-python-magic-of-rpy2-d8cbbf991571

* https://rpy2.github.io/doc/v3.0.x/html/generated_rst/pandas.html

In [ ]:
%%file get_sentimentr.R

library(sentimentr)
library(lexicon)

get_sentimentr_values <- function(s_v) {
  
  print('Processing sentimentr_jockersrinker')
  sentimentr_jockersrinker <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers_rinker, 
                                        hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                        adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  print('Processing sentimentr_jockers')
  sentimentr_jockers <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers, 
                                        hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                        adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  print('Processing sentimentr_huliu')
  sentimentr_huliu <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_huliu, 
                                        hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                        adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  print('Processing sentimentr_nrc')
  sentimentr_nrc <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_nrc, 
                                        hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                        adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  print('Processing sentimentr_senticnet')
  sentimentr_senticnet <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_senticnet, 
                                        hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                        adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  print('Processing sentimentr_sentiword')
  sentimentr_sentiword <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_sentiword, 
                                        hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                        adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  print('Processing sentimentr_loughran_mcdonald')
  sentimentr_loughran_mcdonald <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_loughran_mcdonald, 
                                        hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                        adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  print('Processing sentimentr_socal_google')
  sentimentr_socal_google <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_socal_google, 
                                        hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                        adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  anovel_sentimentr_df <- data.frame('text_clean' = s_v,
                                'sentimentr_jockersrinker' = sentimentr_jockersrinker$sentiment,
                                'sentimentr_jockers' = sentimentr_jockers$sentiment,
                                'sentimentr_huliu' = sentimentr_huliu$sentiment,
                                'sentimentr_nrc' = sentimentr_nrc$sentiment,
                                'sentimentr_senticnet' = sentimentr_senticnet$sentiment,
                                'sentimentr_sentiword' = sentimentr_sentiword$sentiment,
                                'sentimentr_loughran_mcdonald' = sentimentr_loughran_mcdonald$sentiment,
                                'sentimentr_socal_google' = sentimentr_socal_google$sentiment
                                )
  return(anovel_sentimentr_df)

}

In [ ]:
!cat get_sentimentr.R

In [ ]:
# Setup python robject with external library::function()
# https://rpy2.github.io/doc/v3.0.x/html/generated_rst/pandas.html

# import rpy2.robjects as robjects

# Defining the R script and loading the instance in Python
# from rpy2.robjects import pandas2ri 
r = robjects.r

# Loading the function we have defined in R.
r['source']('get_sentimentr.R')

# Reading and processing data
get_sentimentr_function_r = robjects.globalenv['get_sentimentr_values']

In [ ]:
# Test

# Convert Python List of Strings to a R vector of characters
test_ls = novels_all_dt['cdickens_achristmascarol']['text_clean'].to_list()
s_v = robjects.StrVector(test_ls)
type(s_v)

get_sentimentr_function_r(s_v)

In [ ]:
novels_dt.keys()

In [ ]:
text_clean_ct = novels_all_dt['dbrown_thedavincicode'].text_clean.isna().sum()
text_clean_ct
# len(text_clean_ls.isnull())

**[RE-EXECUTE] May have to re-execute following code cell several times**

In [ ]:
%%time

# NOTE: 8m19s 13 Novels 
#      16m39s 19 Novels
#     -----------------
#      24m58s 32 Novels

#      10m58s 10 Novels 14:39 at 20220115
#             32 Novels

# Call external get_sentimentr::get_sentimentr_values with Python loop over all novels

# novels_sentimentr_df = {}

anovel_df = pd.DataFrame()

novels_keys_ls = list(novels_all_dt.keys())
novels_keys_ls.sort()
# for i, anovel in enumerate(novels_keys_ls[:19]):
for i, anovel in enumerate(novels_keys_ls):  
  print(f'\nNovel #{i}: {anovel}')
  print(f'     {novels_all_dt[anovel].shape}')
  # Get text_clean as list of strings
  text_clean_ls = novels_all_dt[anovel]['text_clean'].to_list()

  # Convert Python List of Strings to a R vector of characters
  # https://rpy2.github.io/doc/v3.0.x/html/generated_rst/pandas.html
  s_v = robjects.StrVector(text_clean_ls)
  anovel_df_r = get_sentimentr_function_r(s_v)

  # Convert rpy2.robjects.vectors.DataFrame to pandas.core.frame.DataFrame
  # https://stackoverflow.com/questions/20630121/pandas-how-to-convert-r-dataframe-back-to-pandas 
  print(f'type(anovel_df_r): {type(anovel_df_r)}')
  anovel_df = pd.DataFrame.from_dict({ key : np.asarray(anovel_df_r.rx2(key)) for key in anovel_df_r.names })
  print(f'type(anovel_df): {type(anovel_df)}')

  # Save Results
  novels_sentimentr_dt[anovel] = anovel_df.copy(deep=True)


In [ ]:
type(novels_sentimentr_df)

### Save Checkpoint

In [ ]:
# Verify save_to directory

subdir_sentiments
print('\n')
!ls $subdir_sentiments

In [ ]:
novels_sentimentr_dt.keys()

In [ ]:
novels_sentimentr_dt['cdickens_achristmascarol'])

In [ ]:
# Save sentiment values to subdir_sentiments

write_dict_dfs(novels_sentimentr_dt, out_file='all_7sentimentr.json', out_dir=subdir_sentiments)

## Plot SentimentR 7 Models

In [ ]:
#@markdown Select option to save plots:
Save_Raw_Plots = True #@param {type:"boolean"}

Save_Smooth_Plots = True #@param {type:"boolean"}
Resolution = "100" #@param ["100", "300"]



In [ ]:
sentimentr_cols_ls = novels_sentimentr_dt['cdickens_achristmascarol'].columns
sentimentr_models_ls = [x for x in sentimentr_cols_ls if 'sentimentr_' in x]
sentimentr_models_ls

# OR

# syuzhetr_models_ls = ['syuzhetr_afinn', 'syuzhetr_bing', 'syuzhetr_nrc', 'syuzhetr_syuzhet']

In [ ]:
# Verify 4 Sentiment Models from Syuzhet for sample Novel

for i, anovel in enumerate(list(novels_all_dt.keys())):

  # Raw Sentiments 
  fig = novels_sentimentr_dt[anovel][sentimentr_models_ls].plot(title=f'{novels_dt[anovel][0]}\n SentimentR 7 Models: Raw Sentiments', alpha=0.3)
  plt.show()

  if Save_Raw_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_sentimentr_raw_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))

  
  # Smoothed Sentiments (SMA 10%)
  # novel_sample = 'cdickens_achristmascarol'
  win_10per = int(novels_sentimentr_dt[anovel].shape[0] * 0.1)
  novels_sentimentr_dt[anovel][sentimentr_models_ls].rolling(win_10per, center=True, min_periods=0).mean().plot(title=f'{novels_dt[anovel][0]}\n SentimentR 7 Models: Smoothed Sentiments (SMA 10%)', alpha=0.3)
  plt.show()

  if Save_Smooth_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_sentimentr_smooth10sma_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))


# **END OF NOTEBOOK**

### Save Checkpoint

In [ ]:
# Verify save_to directory

subdir_sentiments
print('\n')
!ls $subdir_sentiments

In [ ]:
# Save sentiment values to subdir_sentiments

write_dict_dfs(novels_syuzhetr_dt, out_file='all_4syuzhetr.json', out_dir=subdir_sentiments)

## Plot SyuzhetR 4 Models

In [ ]:
#@markdown Select option to save plots:
Save_Raw_Plots = True #@param {type:"boolean"}

Save_Smooth_Plots = True #@param {type:"boolean"}
Resolution = "100" #@param ["100", "300"]



In [ ]:
syuzhetr_cols_ls = novels_syuzhetr_dt['cdickens_achristmascarol'].columns
syuzhetr_models_ls = [x for x in syuzhetr_cols_ls if 'syuzhetr_' in x]
syuzhetr_models_ls

# OR

# syuzhetr_models_ls = ['syuzhetr_afinn', 'syuzhetr_bing', 'syuzhetr_nrc', 'syuzhetr_syuzhet']

In [ ]:
novels_dt['cdickens_achristmascarol'][0]

In [ ]:
subdir_plots_arcs

In [ ]:
# Verify 4 Sentiment Models from Syuzhet for sample Novel

for i, anovel in enumerate(list(novels_all_dt.keys())):

  # Raw Sentiments 
  fig = novels_syuzhetr_dt[anovel][syuzhetr_models_ls].plot(title=f'{novels_dt[anovel][0]}\n SyuzhetR 4 Models: Raw Sentiments', alpha=0.3)
  plt.show()

  if Save_Raw_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_syuzhetr_raw_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))

  
  # Smoothed Sentiments (SMA 10%)
  # novel_sample = 'cdickens_achristmascarol'
  win_10per = int(novels_syuzhetr_dt[anovel].shape[0] * 0.1)
  novels_syuzhetr_dt[anovel][syuzhetr_models_ls].rolling(win_10per, center=True, min_periods=0).mean().plot(title=f'{novels_dt[anovel][0]}\n SyuzhetR 4 Models: Smoothed Sentiments (SMA 10%)', alpha=0.3)
  plt.show()

  if Save_Smooth_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_syuzhetr_smooth10sma_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))


### Save Checkpoint

In [ ]:
# Verify save_to directory

subdir_sentiments
print('\n')
!ls $subdir_sentiments

In [ ]:
# Save sentiment values to subdir_sentiments

write_dict_dfs(novels_syuzhetr_dt, out_file='all_4syuzhetr.json', out_dir=subdir_sentiments)

## Plot SyuzhetR 4 Models

In [ ]:
#@markdown Select option to save plots:
Save_Raw_Plots = True #@param {type:"boolean"}

Save_Smooth_Plots = True #@param {type:"boolean"}
Resolution = "100" #@param ["100", "300"]



In [ ]:
syuzhetr_cols_ls = novels_syuzhetr_dt['cdickens_achristmascarol'].columns
syuzhetr_models_ls = [x for x in syuzhetr_cols_ls if 'syuzhetr_' in x]
syuzhetr_models_ls

# OR

# syuzhetr_models_ls = ['syuzhetr_afinn', 'syuzhetr_bing', 'syuzhetr_nrc', 'syuzhetr_syuzhet']

In [ ]:
novels_dt['cdickens_achristmascarol'][0]

In [ ]:
subdir_plots_arcs

In [ ]:
# Verify 4 Sentiment Models from Syuzhet for sample Novel

for i, anovel in enumerate(list(novels_all_dt.keys())):

  # Raw Sentiments 
  fig = novels_syuzhetr_dt[anovel][syuzhetr_models_ls].plot(title=f'{novels_dt[anovel][0]}\n SyuzhetR 4 Models: Raw Sentiments', alpha=0.3)
  plt.show()

  if Save_Raw_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_syuzhetr_raw_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))

  
  # Smoothed Sentiments (SMA 10%)
  # novel_sample = 'cdickens_achristmascarol'
  win_10per = int(novels_syuzhetr_dt[anovel].shape[0] * 0.1)
  novels_syuzhetr_dt[anovel][syuzhetr_models_ls].rolling(win_10per, center=True, min_periods=0).mean().plot(title=f'{novels_dt[anovel][0]}\n SyuzhetR 4 Models: Smoothed Sentiments (SMA 10%)', alpha=0.3)
  plt.show()

  if Save_Smooth_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_syuzhetr_smooth10sma_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))


### Save Checkpoint

In [ ]:
# Verify save_to directory

subdir_sentiments
print('\n')
!ls $subdir_sentiments

In [ ]:
# Save sentiment values to subdir_sentiments

write_dict_dfs(novels_syuzhetr_dt, out_file='all_4syuzhetr.json', out_dir=subdir_sentiments)

## Plot SyuzhetR 4 Models

In [ ]:
#@markdown Select option to save plots:
Save_Raw_Plots = True #@param {type:"boolean"}

Save_Smooth_Plots = True #@param {type:"boolean"}
Resolution = "100" #@param ["100", "300"]



In [ ]:
syuzhetr_cols_ls = novels_syuzhetr_dt['cdickens_achristmascarol'].columns
syuzhetr_models_ls = [x for x in syuzhetr_cols_ls if 'syuzhetr_' in x]
syuzhetr_models_ls

# OR

# syuzhetr_models_ls = ['syuzhetr_afinn', 'syuzhetr_bing', 'syuzhetr_nrc', 'syuzhetr_syuzhet']

In [ ]:
novels_dt['cdickens_achristmascarol'][0]

In [ ]:
subdir_plots_arcs

In [ ]:
# Verify 4 Sentiment Models from Syuzhet for sample Novel

for i, anovel in enumerate(list(novels_all_dt.keys())):

  # Raw Sentiments 
  fig = novels_syuzhetr_dt[anovel][syuzhetr_models_ls].plot(title=f'{novels_dt[anovel][0]}\n SyuzhetR 4 Models: Raw Sentiments', alpha=0.3)
  plt.show()

  if Save_Raw_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_syuzhetr_raw_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))

  
  # Smoothed Sentiments (SMA 10%)
  # novel_sample = 'cdickens_achristmascarol'
  win_10per = int(novels_syuzhetr_dt[anovel].shape[0] * 0.1)
  novels_syuzhetr_dt[anovel][syuzhetr_models_ls].rolling(win_10per, center=True, min_periods=0).mean().plot(title=f'{novels_dt[anovel][0]}\n SyuzhetR 4 Models: Smoothed Sentiments (SMA 10%)', alpha=0.3)
  plt.show()

  if Save_Smooth_Plots:
    plt.savefig(f'{subdir_plots_arcs}plot_syuzhetr_smooth10sma_{anovel}_dpi{Resolution}.png', dpi=int(Resolution))


In [ ]:
%%R -i s_v -i novels_roots -o novels_sentimentr_df

# novels_key_ls = c('cdickens_achristmascarol')

get_sentimentr_vals = function()
print('Processing sentimentr_jockersrinker')
sentimentr_jockersrinker <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers_rinker, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_jockers')
sentimentr_jockers <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_huliu')
sentimentr_huliu <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_huliu, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_nrc')
sentimentr_nrc <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_nrc, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_senticnet')
sentimentr_senticnet <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_senticnet, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_sentiword')
sentimentr_sentiword <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_sentiword, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_loughran_mcdonald')
sentimentr_loughran_mcdonald <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_loughran_mcdonald, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_socal_google')
sentimentr_socal_google <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_socal_google, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

novels_sentimentr_df <- data.frame('text_clean' = s_v,
                              'sentimentr_jockersrinker' = sentimentr_jockersrinker$sentiment,
                              'sentimentr_jockers' = sentimentr_jockers$sentiment,
                              'sentimentr_huliu' = sentimentr_huliu$sentiment,
                              'sentimentr_nrc' = sentimentr_nrc$sentiment,
                              'sentimentr_senticnet' = sentimentr_senticnet$sentiment,
                              'sentimentr_sentiword' = sentimentr_sentiword$sentiment,
                              'sentimentr_loughran_mcdonald' = sentimentr_loughran_mcdonald$sentiment,
                              'sentimentr_socal_google' = sentimentr_socal_google$sentiment
                              )


In [ ]:
%%R -i s_v -i novels_roots -o novels_sentimentr_df

# novels_key_ls = c('cdickens_achristmascarol')


print('Processing sentimentr_jockersrinker')
sentimentr_jockersrinker <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers_rinker, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_jockers')
sentimentr_jockers <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_huliu')
sentimentr_huliu <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_huliu, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_nrc')
sentimentr_nrc <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_nrc, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_senticnet')
sentimentr_senticnet <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_senticnet, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_sentiword')
sentimentr_sentiword <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_sentiword, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_loughran_mcdonald')
sentimentr_loughran_mcdonald <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_loughran_mcdonald, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

print('Processing sentimentr_socal_google')
sentimentr_socal_google <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_socal_google, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

novels_sentimentr_df <- data.frame('text_clean' = s_v,
                              'sentimentr_jockersrinker' = sentimentr_jockersrinker$sentiment,
                              'sentimentr_jockers' = sentimentr_jockers$sentiment,
                              'sentimentr_huliu' = sentimentr_huliu$sentiment,
                              'sentimentr_nrc' = sentimentr_nrc$sentiment,
                              'sentimentr_senticnet' = sentimentr_senticnet$sentiment,
                              'sentimentr_sentiword' = sentimentr_sentiword$sentiment,
                              'sentimentr_loughran_mcdonald' = sentimentr_loughran_mcdonald$sentiment,
                              'sentimentr_socal_google' = sentimentr_socal_google$sentiment
                              )


In [ ]:
novels_sentimentr_df.head()

In [ ]:
# Get list of sentimentr models from columns

sentimentr_models_ls = [x for x in novels_sentimentr_df.columns if 'sentimentr_' in x]
sentimentr_models_ls

In [ ]:
# Verify 4 Sentiment Models from Syuzhet for sample Novel

# Raw Sentiments 
# novels_all_dt['cdickens_achristmascarol'][['syuzhetr_syuzhet','syuzhetr_bing','syuzhetr_afinn','syuzhetr_nrc']].plot(title='Raw Sentiments', alpha=0.3)
novels_sentimentr_df[sentimentr_models_ls].plot(title='SentimentR Raw Sentiments', alpha=0.3)

# Smoothed Sentiments (SMA 10%)
novel_sample = 'cdickens_achristmascarol'
win_10per = int(novels_all_dt['cdickens_achristmascarol'].shape[0] * 0.1)
novels_sentimentr_df[sentimentr_models_ls].rolling(win_10per, center=True, min_periods=0).mean().plot(title='SentimentR Smoothed Sentiments (SMA 10%)', alpha=0.3)
plt.show()


In [ ]:
%%file get_sentimentr.R

library(sentimentr)
library(lexicon)

get_sentimentr_values <- function(df_name, col_name){
  #' Preprocessing df to filter country
  #'
  #' This function returns a subset of the df
  #' if the value of the country column contains 
  #' the country we are passing
  #'
  #' @param df The dataframe containing the data 
  #' @param country The country we want to filter
  #
  print('Processing sentimentr_jockers')
  sentimentr_jockers <- sentiment(df_name$col_name, polarity_dt=lexicon::hash_sentiment_jockers, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)



  return(sentimentr_jockers)
}

In [ ]:
!cat get_sentimentr.R

In [ ]:
df = novels_all_dt['cdickens_achristmascarol']
df.head()

In [ ]:
dir(pandas2ri)

In [ ]:
# https://medium.com/analytics-vidhya/calling-r-from-python-magic-of-rpy2-d8cbbf991571

# import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri# Defining the R script and loading the instance in Python
from rpy2.robjects.conversion import localconverter

r = robjects.r
r['source']('get_sentimentr.R')# Loading the function we have defined in R.
sentimentr_function_r = robjects.globalenv['get_sentimentr_values']

# r['source']('sentimentr')# Loading the function we have defined in R.
# sentiment_function_r = robjects.globalenv['sentiment']

# Reading and processing data
# df = pd.read_csv("Country-Sales.csv")#converting it into r object for passing into r function
df = novels_all_dt['cdickens_achristmascarol']
with localconverter(robjects.default_converter + pandas2ri.converter):
  df_r = robjects.conversion.py2rpy(df)

# df_r = pandas2ri.py2rpy_dataframe(df)

#Invoking the R function and getting the result
df_result_r = sentimentr_function_r(df_r, 'text_clean')

#Converting it back to a pandas dataframe.
df_result = pandas2ri.py2ri(df_result_r)

df_result.head()



## Save Checkpoint


# **END OF NOTEBOOK**

In [ ]:
pd.DataFrame(syuzhetr_syuzhet).plot()

In [ ]:
install.packages('syuzhet')
library(syuzhet)

In [ ]:
files <- list.files(pattern="*.csv", full.names=TRUE, recursive=FALSE)
length(files)

In [ ]:
files[1]

In [ ]:
sentiment_syuzhet <- get_sentiment(s_v, method='syuzhet')
sentiment_syuzhet

In [ ]:
get_syuzhetr_sentiments <- function(afilename) {
  anovel_df <- read.csv(file = afilename, header=FALSE)
  # typeof(anovel_df$V3)
  # return(anovel_df)
  s_v <- anovel_df$V3

  syuzhetr_jockers <- get_sentiment(s_v, method='syuzhet')
  syuzhetr_bing <- get_sentiment(s_v, method='bing')
  syuzhetr_afinn <- get_sentiment(s_v, method='afinn')
  syuzhetr_nrc <- get_sentiment(s_v, method='nrc')

  syuzhet_df <- data.frame(syuzhetr_jockers,
                           syuzhetr_bing,
                           syuzhetr_afinn,
                           syuzhetr_nrc)

  return(syuzhet_df)
}

In [ ]:
get_syuzhetr_sentiments(files[1])

In [ ]:
get_syuzhetr_sentiments()

In [ ]:
lapply(files, function(x) {
  print(paste0('Novel: ', x))
  anovel_df <- read.csv(file = x, header=FALSE)
  syuzhetr_df <- get_syuzhetr_sentiments(anovel_df)
  head(anovel_df)
})

## Syuzhet Lexicon

In [ ]:
sentiment_syuzhet <- get_sentiment(s_v, method='syuzhet')
sentiment_syuzhet

In [ ]:
simple_plot(sentiment_syuzhet)

In [ ]:
s_v_sentiment_dct <- get_dct_transform(
  sentiment_syuzhet,
  low_pass_size = 5,
  x_reverse_len = 100,
  scale_vals = F,
  scale_range = T
)
plot(
  s_v_sentiment_dct,
  type = 'l',
  main="DCT Transformed Syuzhet Sentiments",
  xlab = "Narrative Time",
  ylab = "Emotional Valence",
  col = "red"
)

## Bing Lexicon

In [ ]:
sentiment_bing <- get_sentiment(s_v, method='bing')
sentiment_bing

In [ ]:
simple_plot(sentiment_bing)

In [ ]:
s_v_sentiment_dct <- get_dct_transform(
  sentiment_bing,
  low_pass_size = 5,
  x_reverse_len = 100,
  scale_vals = F,
  scale_range = T
)
plot(
  s_v_sentiment_dct,
  type = 'l',
  main="DCT Transformed Bing Sentiments",
  xlab = "Narrative Time",
  ylab = "Emotional Valence",
  col = "red"
)

## AFINN Lexicon

In [ ]:
sentiment_afinn <- get_sentiment(s_v, method='afinn')
sentiment_afinn

In [ ]:
simple_plot(sentiment_afinn)

In [ ]:
s_v_sentiment_dct <- get_dct_transform(
  sentiment_afinn,
  low_pass_size = 5,
  x_reverse_len = 100,
  scale_vals = F,
  scale_range = T
)
plot(
  s_v_sentiment_dct,
  type = 'l',
  main="DCT Transformed AFINN Sentiments",
  xlab = "Narrative Time",
  ylab = "Emotional Valence",
  col = "red"
)

## NRC Lexicon

In [ ]:
sentiment_nrc <- get_sentiment(s_v, method='nrc')
sentiment_nrc

In [ ]:
simple_plot(sentiment_nrc)

In [ ]:
s_v_sentiment_dct <- get_dct_transform(
  sentiment_nrc,
  low_pass_size = 5,
  x_reverse_len = 100,
  scale_vals = F,
  scale_range = T
)
plot(
  s_v_sentiment_dct,
  type = 'l',
  main="DCT Transformed Bing Sentiments",
  xlab = "Narrative Time",
  ylab = "Emotional Valence",
  col = "red"
)

## Stanford Lexicon (Disabled, req Java OpenNLP)

In [ ]:
sentiment_stanford <- get_sentiment(s_v, method='stanford')
sentiment_stanford

In [ ]:
simple_plot(sentiment_nrc)

In [ ]:
s_v_sentiment_dct <- get_dct_transform(
  sentiment_nrc,
  low_pass_size = 5,
  x_reverse_len = 100,
  scale_vals = F,
  scale_range = T
)
plot(
  s_v_sentiment_dct,
  type = 'l',
  main="DCT Transformed Bing Sentiments",
  xlab = "Narrative Time",
  ylab = "Emotional Valence",
  col = "red"
)

## Combine all SyuzhetR Lexicons

In [ ]:
syuzhetr_all_df <- data.frame(syuzhet = sentiment_syuzhet,
                              bing = sentiment_bing,
                              afinn = sentiment_afinn,
                              nrc = sentiment_nrc)

syuzhetr_all_df

In [ ]:
write.csv(syuzhetr_all_df, 'syuzhetr_novel.csv', row.names=FALSE)

# SentimentR Library

* https://github.com/trinker/sentimentr

* https://cran.r-project.org/web/packages/sentimentr/

In [ ]:
install.packages('sentimentr')
library(sentimentr)

In [ ]:
if (!require("pacman")) install.packages("pacman")
pacman::p_load_gh("trinker/sentimentr", "trinker/stansent", "sfeuerriegel/SentimentAnalysis", "wrathematics/meanr")
pacman::p_load(syuzhet, qdap, microbenchmark, RSentiment)

In [ ]:
files <- list.files(pattern="*.csv", full.names=TRUE, recursive=FALSE)
length(files)

In [ ]:
files[1]

In [ ]:
sentiment_syuzhet <- get_sentiment(s_v, method='syuzhet')
sentiment_syuzhet

In [ ]:
get_sentimentr_sentiments <- function(afilename) {
  anovel_df <- read.csv(file = afilename, header=FALSE)
  # typeof(anovel_df$V3)
  # return(anovel_df)
  s_v <- anovel_df$V3

  jockersrinker <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers_rinker, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  jockers <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  huliu <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_huliu, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  loughran_mcdonald <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_loughran_mcdonald, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  sentimentr_nrc <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_nrc, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  senticnet <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_senticnet, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

  sentiword <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_sentiword, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)                                                                                                 

  socal_google <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_socal_google, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)                                                                                                 

  slangsd <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_slangsd, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)                                                                                                 

  emojis <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_emojis, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)                                                                                                 

  sentimentr_df <- data.frame(element_id = jockersrinker$element_id,
                              sentence_id = jockersrinker$sentence_id,
                              word_count = jockersrinker$word_count,
                              jockers_rinker = jockersrinker$sentiment,
                              jockers = jockers$sentiment,
                              huliu = huliu$sentiment,
                              lmcd = loughran_mcdonald$sentiment,
                              nrc = sentimentr_nrc$sentiment,
                              senticnet = senticnet$sentiment,
                              sentiword = sentiword$sentiment,
                              socal_google = socal_google$sentiment,
                              slangsd = slangsd$sentiment,
                              emojis = emojis$sentiment)
  
  return(sentimentr_df)
}

In [ ]:
files[1]

In [ ]:
get_sentimentr_sentiments(files[1])

In [ ]:
typeof(s_v)

In [ ]:
ase <- c(
    "I haven't been sad in a long time.",
    "I am extremely happy today.",
    "It's a good day.",
    "But suddenly I'm only a little bit happy.",
    "Then I'm not happy at all.",
    "In fact, I am now the least happy person on the planet.",
    "There is no happiness left in me.",
    "Wait, it's returned!",
    "I don't feel so bad after all!"
)
typeof(ase)

In [ ]:
syuzhet <- setNames(as.data.frame(lapply(c("syuzhet", "bing", "afinn", "nrc"),
    function(x) get_sentiment(s_v, method=x))), c("jockers", "bing", "afinn", "nrc"))


In [ ]:
dim(syuzhet)

In [ ]:
syuzhet

## Jockers_Rinker Lexicon

In [ ]:
SentimentAnalysis <- apply(analyzeSentiment(s_v)[c('SentimentGI', 'SentimentLM', 'SentimentQDAP') ], 2, round, 2)
colnames(SentimentAnalysis) <- gsub('^Sentiment', "SA_", colnames(SentimentAnalysis))

In [ ]:
sentimentr_jockersrinker <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers_rinker, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_jockersrinker

## Jockers Lexicon

In [ ]:
sentimentr_jockers <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_jockers, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_jockers

## Hu_Liu Lexicon

In [ ]:
sentimentr_huliu <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_huliu, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_huliu

## Loughran_McDonald Lexicon

In [ ]:
sentimentr_loughranmcdonald <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_loughran_mcdonald, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_loughranmcdonald

## NRC Lexicon

In [ ]:
sentimentr_nrc <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_nrc, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_nrc

## SenticNet Lexicon

In [ ]:
sentimentr_senticnet <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_senticnet, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_senticnet

## SentiWord Lexicon

In [ ]:
sentimentr_sentiword <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_sentiword, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_sentiword

## Socal_Google Lexicon

In [ ]:
sentimentr_socalgoogle <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_socal_google, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_socalgoogle

## SlangSD Lexicon

In [ ]:
sentimentr_slangsd <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_slangsd, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_slangsd

## Emoji Lexicon

In [ ]:
sentimentr_emojis <- sentiment(s_v, polarity_dt=lexicon::hash_sentiment_emojis, 
                                      hypen="", amplifier.weight=0.8, n.before=5, n.after=2,
                                      adversative.weight=0.25, neutral.nonverb.like = FALSE, missing_value = 0)

In [ ]:
sentimentr_emojis

## Combine all SentimentR Lexicons

In [ ]:
sentimentr_all_df <- data.frame(element_id = sentimentr_jockersrinker$element_id,
                               sentence_id = sentimentr_jockersrinker$sentence_id,
                               word_count = sentimentr_jockersrinker$word_count,
                               jockersrinker = sentimentr_jockersrinker$sentiment,
                               jockers = sentimentr_jockers$sentiment,
                               huliu = sentimentr_huliu$sentiment,
                               nrc = sentimentr_nrc$sentiment,
                               senticnet = sentimentr_senticnet$sentiment,
                               sentiword = sentimentr_sentiword$sentiment,
                               loughranmcdonald = sentimentr_loughranmcdonald$sentiment,
                               socalgoogle = sentimentr_socalgoogle$sentiment,
                               slangsd = sentimentr_slangsd$sentiment,
                               emojis = sentimentr_emojis$sentiment)

sentimentr_all_df

In [ ]:
write.csv(sentimentr_all_df, 'sentiment_novel.csv', row.names=FALSE)

# Combine SentimentR and SyuzhetR

In [ ]:
sentiments_all_df <- merge(syuzhetr_all_df, sentimentr_all_df)
sentiments_all_df

In [ ]:
write.csv(sentiments_all_df, 'sentiments_novel.csv', row.names=FALSE)

# Iterate over all files in the Directory

In [ ]:
getwd()

In [ ]:
list.files()

In [ ]:
subdir_preprocessed

In [ ]:
list.files(pattern="*.csv")

In [ ]:
# files <- list.files(path=subdir_preprocessed, pattern="*.csv", full.names=TRUE, recursive=FALSE)
files <- list.files(pattern="*.csv", full.names=TRUE, recursive=FALSE)
length(files)

In [ ]:
files[1]

In [ ]:
(strsplit(files[1], '[/.]')[[1]])[3]

In [ ]:
for (afile in files) {
  # afile = .
  file_root = (strsplit(afile, '[/.]')[[1]])[3]
  file_roots_v <- c(file_roots_v, file_root)
}

In [ ]:
length(file_roots_v)

In [ ]:
library(stringr)

In [ ]:
tibble(files) %>%
  separate(files, into = c("lv1", "lv2", "lv3"), sep = "/", fill = "left") %>%
  mutate("version" = str_extract(lv3, regex("v\\d+")))

In [ ]:
files[0]

In [ ]:
lapply(files, function(x) {
  print(paste0('\nNovel: ', x))
  # anovel_df <- read.csv(file = x, header=FALSE)
  syuzhetr_df <- get_syuzhetr_sentiments(x)
  sentimentr_df <- get_sentimentr_sentiments(x)

  anovel_sentiments_df <- merge(sentimentr_df, syuzhetr_df)
  # head(anovel_sentiments_df, 5)

  file_root = (strsplit(x, '[/.]')[[1]])[3]
  outfile <- paste0('sentiments_', file_root, '.csv')
  print(paste0('  saving to: ', outfile))
  write.csv(x=anovel_sentiments_df, file=outfile, row.names=FALSE)
})

In [ ]:
lapply(files, function(x) {
  carSpeeds <- read.csv(file = 'data/car-speeds.csv')
  head(carSpeeds)

  
    t <- read.table(x, header=TRUE) # load file
    # apply function
    out <- function(t)
    # write to file
    write.table(out, "path/to/output", sep="\t", quote=FALSE, row.names=FALSE, col.names=TRUE)
})

# **END OF NOTEBOOK**

In [ ]:
sentiment_all <- left_just(data.frame(
    sentimentr_jockersrinker = round(sentiment(s_v, question.weight = 0)[["sentiment"]], 2),
    sentimentr_jockers = round(sentiment(s_v, lexicon::hash_sentiment_jockers, question.weight = 0)[["sentiment"]], 2),    
    sentimentr_huliu = round(sentiment(s_v, lexicon::hash_sentiment_huliu, question.weight = 0)[["sentiment"]], 2),    
    sentimentr_sentiword = round(sentiment(s_v, lexicon::hash_sentiment_sentiword, question.weight = 0)[["sentiment"]], 2),    
))

In [ ]:
left_just(data.frame(
    # stanford = sentiment_stanford(s_v)[["sentiment"]],
    sentimentr_jockersrinker = round(sentiment(s_v, question.weight = 0)[["sentiment"]], 2),
    sentimentr_jockers = round(sentiment(s_v, lexicon::hash_sentiment_jockers, question.weight = 0)[["sentiment"]], 2),    
    sentimentr_huliu = round(sentiment(s_v, lexicon::hash_sentiment_huliu, question.weight = 0)[["sentiment"]], 2),    
    sentimentr_sentiword = round(sentiment(s_v, lexicon::hash_sentiment_sentiword, question.weight = 0)[["sentiment"]], 2),    
    RSentiment = calculate_score(s_v), 
    SentimentAnalysis,
    meanr = score(s_v)[['score']],
    syuzhet,
    sentences = s_v,
    stringsAsFactors = FALSE
), "sentences")

In [ ]:
os <- import("os")
os$listdir(".")

In [ ]:
typeof(novel_df['text_clean'])

In [ ]:
print(novel_df['text_clean'])

In [ ]:
(novel_df['text_clean'])

In [ ]:
library(stringi)

In [ ]:
novel_str = stri_join_list(novel_df['text_clean'], sep=' ', collapse=TRUE)
typeof(novel_str)

In [ ]:
substr(novel_str, 1, 100)

In [ ]:
bovary_s_v <- get_sentences(bovary_str)
bovary_s_v

In [ ]:
typeof(bovary_s_v)

In [ ]:
bovary_sentiment <- get_sentiment(bovary_s_v)

In [ ]:
typeof(bovary_sentiment)

In [ ]:

simple_plot(bovary_sentiment)

## Installing libraries

In [ ]:
install.packages('reticulate')

In [ ]:
# which version of python are we using, where is it?

cli_msg <- system('which python', intern=TRUE)
cli_msg

In [ ]:
# in RStudio, type 'usethis::edit_r_profile()' add this line

Sys.setenv(RETICULATE_PYTHON = "/usr/local/bin/python")

library(reticulate)

In [ ]:
print("hello")

In [ ]:
repl_python(input='print("hello")')

In [ ]:
system('wget https://raw.githubusercontent.com/mwaskom/seaborn-data/master/flights.csv', intern=TRUE)


In [ ]:
cli_msg <- system('ls -altr', intern=TRUE)
cli_msg

In [ ]:
os <- import("os")
os$listdir(".")

In [ ]:
#importing required Python libraries/modules
sns <- import('seaborn')
plt <- import('matplotlib.pyplot')
pd <- import('pandas')

# Syuzhet

In [ ]:
install.packages('syuzhet')

In [ ]:
library(syuzhet)

In [ ]:
install.packages('gutenbergr')

In [ ]:
library(gutenbergr)

In [ ]:
library(dplyr)

In [ ]:
gutenberg_metadata %>%
  filter(title == "Wuthering Heights")

In [ ]:
gutenberg_works(author == "Austen, Jane")

In [ ]:
library(stringr)
gutenberg_works(str_detect(author, "Austen"))

In [ ]:
library(stringr)
gutenberg_works(str_detect(title, "Bovary"))

In [ ]:
bovary <- gutenberg_download(2413)
bovary

In [ ]:
os <- import("os")
os$listdir(".")

In [ ]:
typeof(bovary[1])

In [ ]:
typeof(bovary[2][0])

In [ ]:
library(stringi)

In [ ]:
bovary_str = stri_join_list(bovary[2], sep=' ', collapse=TRUE)
typeof(bovary_str)

In [ ]:
substr(bovary_str, 1, 100)

In [ ]:
s_v <- get_sentences(bovary_str)
s_v

In [ ]:
s_v_sentiment <- get_sentiment(s_v)
s_v_sentiment

In [ ]:
simple_plot(s_v_sentiment)

In [ ]:
bovary_str = str_flatten(bovary[2], " ")

In [ ]:
bovary_str2 = str_flatten(noquote(bovary[2]), " ")

In [ ]:
bovary_str2

In [ ]:
typeof(bovary_str)

In [ ]:
bovary_str

In [ ]:
bovary_v <- get_sentences(bovary_str)

In [ ]:
typeof(bovary_v)

In [ ]:
sent_v <- get_sentences(bovary)

In [ ]:
```{python}
import os
```

In [ ]:
#using R's inbuilt AirPassengers dataset
df <- datasets::AirPassengers

#converting Time-Series object into an R Dataframe 
#Thx: https://stackoverflow.com/questions/5331901/transforming-a-time-series-into-a-data-frame-and-back
df1 <- data.frame(tapply(df, list(year = floor(time(df)), month = month.abb[cycle(df)]), c))
df1 <- df1[month.abb]

#building a heatmap using seaborn 
#please note the function r_to_py() that converts R object into a python 
sns$heatmap(r_to_py(df1), fmt="g", cmap ='viridis')

#display the plot
plt$show()

In [ ]:
```{python}

name = 'Bill'

print(f'Hello {name}!')
```

In [ ]:
```{python}
import pandas as pd

flights = pd.read_csv('flights.csv')
flights = flights[flights['dest'] == "ORD"]
flights = flights[['carrier', 'dep_delay', 'arr_delay']]
flights = flights.dropna()
```

In [ ]:
# using R's inbuilt AirPassengers dataset
df <- datasets::AirPassengers

In [ ]:
df

In [ ]:
install.packages('caret')

In [ ]:
install.packages('mlbench')

## Importing libraries

In [ ]:
library(caret)

In [ ]:
library(ggplot2)

In [ ]:
library(mlbench)

## How many CPU cores are there?

In [ ]:
library(parallel)
detectCores(all.tests = FALSE, logical = TRUE)

---

# Machine Learning in R: Building a Linear Regression Model

YouTube:
https://www.youtube.com/watch?v=el8xP38SWdk

GitHub:
https://github.com/dataprofessor/code/blob/master/linear-regression/boston-housing-linear-regression.R

In [ ]:
############################################
# Data Professor                           #
# http://youtube.com/dataprofessor         #
# http://github.com/dataprofessor          #
# http://facebook.com/dataprofessor        #
# https://www.instagram.com/data.professor #
############################################

# Importing libraries
library(mlbench) # Contains several benchmark data sets (especially the Boston Housing dataset)
library(caret) # Package for machine learning algorithms / CARET stands for Classification And REgression Training

# Importing the Boston Housing data set
data(BostonHousing)

head(BostonHousing)

# Check to see if there are missing data?
sum(is.na(BostonHousing))

# To achieve reproducible model; set the random seed number
set.seed(100)

# Performs stratified random split of the data set
TrainingIndex <- createDataPartition(BostonHousing$medv, p=0.8, list = FALSE)
TrainingSet <- BostonHousing[TrainingIndex,] # Training Set
TestingSet <- BostonHousing[-TrainingIndex,] # Test Set


###############################

# Build Training model
Model <- train(medv ~ ., data = TrainingSet,
               method = "lm",
               na.action = na.omit,
               preProcess=c("scale","center"),
               trControl= trainControl(method="none")
)

# Apply model for prediction
Model.training <-predict(Model, TrainingSet) # Apply model to make prediction on Training set
Model.testing <-predict(Model, TestingSet) # Apply model to make prediction on Testing set

# Model performance (Displays scatter plot and performance metrics)
  # Scatter plot of Training set
plot(TrainingSet$medv,Model.training, col = "blue" )
plot(TestingSet$medv,Model.testing, col = "blue" )

---

# Machine Learning in R: Building a Linear Regression Model

YouTube:
https://www.youtube.com/watch?v=el8xP38SWdk

GitHub:
https://github.com/dataprofessor/code/blob/master/linear-regression/boston-housing-linear-regression.R

### Importing libraries

In [ ]:
library(mlbench) # Contains several benchmark data sets (especially the Boston Housing dataset)
library(caret) # Package for machine learning algorithms / CARET stands for Classification And REgression Training

### Importing the Boston Housing data set

In [ ]:
data(BostonHousing)

head(BostonHousing)

### Check to see if there are missing data?

In [ ]:
sum(is.na(BostonHousing))

### To achieve reproducible model; set the random seed number

In [ ]:
set.seed(100)

### Performs stratified random split of the data set

In [ ]:
TrainingIndex <- createDataPartition(BostonHousing$medv, p=0.8, list = FALSE)
TrainingSet <- BostonHousing[TrainingIndex,] # Training Set
TestingSet <- BostonHousing[-TrainingIndex,] # Test Set

### Build Training model

In [ ]:
Model <- train(medv ~ ., data = TrainingSet,
               method = "lm",
               na.action = na.omit,
               preProcess=c("scale","center"),
               trControl= trainControl(method="none")
)

### Apply model for prediction

In [ ]:
Model.training <-predict(Model, TrainingSet) # Apply model to make prediction on Training set
Model.testing <-predict(Model, TestingSet) # Apply model to make prediction on Testing set


### Model performance (Displays scatter plot and performance metrics)
Scatter plot of Training set

In [ ]:
plot(TrainingSet$medv,Model.training, col = "blue" )

Scatter plot of Testing set

In [ ]:
plot(TestingSet$medv,Model.testing, col = "blue" )